# Getting Started with GlassFlow

This Jupyter Notebook guide provides a hands-on approach to understanding how to get started with GlassFlow. **Create your first pipeline** and interact with the pipeline via the [Python SDK](https://pysdk.docs.glassflow.dev/). By following the steps, you will be able to publish and consume data efficiently from the new pipeline in your local environment.

## Prerequisites
Before starting, ensure you have:

- A GlassFlow account. [Sign up here](http://app.glassflow.dev/) if you don't have one.
- A personal access token from the GlassFlow account you just created. You can get your's in the [profile section](https://app.glassflow.dev/profile) of your account
- Python 3.x installed on your system.
- [Pip](https://pip.pypa.io/en/stable/installation/) installed to manage project packages.

In [ ]:
%pip install "glassflow>=2.0.5"

In [10]:
import glassflow
import os

In [11]:
# Please edit this variable with your own personal access token from https://app.glassflow.dev/profile

glassflow_personal_access_token = ""

## Creating a Pipeline
A pipeline is how you interact with GlassFlow. A pipeline consists of a **Transform function** that is automatically run by GlassFlow for every event entering the pipeline. GlassFlow pipelines ingest JSON data and can do so from multiple sources. The transformed data can then be consumed from the pipeline or automatically sent to data sinks.

In this guide, we are going to set up a pipeline with a basic transform function and send/receive data from the pipeline via the Python SDK.


### Define a basic echo transform function to use with GlassFlow

The transform function is a Python function that you want the GlassFlow pipeline to execute for every event that enters the pipeline. It follows a basic structure with a `handler` method, which serves as the entry point for the function.

In [12]:
%%bash
cat <<EOT > transform.py 
import json
# Write a mandatory 'handler' function
def handler(data, log):
    data["transformed_by"] = "glassflow"
    return data
EOT

In [13]:
client = glassflow.GlassFlowClient(personal_access_token=glassflow_personal_access_token)

In [14]:
# create a space  - Spaces are a way to organize pipelines within glassflow

In [15]:
example_space = client.create_space(name="examples")

In [16]:
# Create a pipeline. A pipeline needs a name, a python code and a space where the pipeline should live

In [17]:
pipeline = client.create_pipeline(name="echo-pipeline", transformation_file="transform.py", 
                                  space_id=example_space.id)

In [18]:
# show the created pipeline id 

In [ ]:
print(pipeline.id)

### Send events to the pipeline

In [20]:
# Create a random data generator to generate some dummy events data
from faker import Faker
def random_datagen():
    fake = Faker()
    return {
        "name": fake.name(),
        "email": fake.email(),
        "id": fake.uuid4()
    }

Each pipeline provides a source and a sink to publish and consume data from the pipeline. We will use those to send and recieve events from the pipeline

In [21]:
data_source = pipeline.get_source()

In [22]:
# Generate some data and send it to the pipeline. Store it locally to compare 
input_data = []
for i in range(100):
    d = random_datagen()
    input_data.append(d)
    r = data_source.publish(d)

In [ ]:
display(input_data)

### Consume events from the pipeline 
Get the sink to consume transformed events from the pipeline

In [24]:
data_sink = pipeline.get_sink()

In [25]:
output_data = []
for i in range(100):
    resp = data_sink.consume()
    output_data.append(resp.json())

In [ ]:
display(output_data)

## Monitor the pipeline

Go to the pipeline logs you created and monitor real-time events.

In [ ]:
## Explore the pipeline logs on the web-UI 
pipeline_url = f"https://app.glassflow.dev/pipelines/{pipeline.id}/logs"
print(pipeline_url)

## Conclusion 

**Congratulations!** You have now set up an event-driven data pipeline entirely in Python using GlassFlow. 
In this getting-started guide we 
- Set up a new pipeline with a basic transform function.
- Published events to the pipeline.
- GlassFlow executed the transform function in real time on the published events.
- Consumed the transformed events back from the pipeline.

As the next step, you can try more examples in the `examples` directory of this repository, or explore the[GlassFlow web-app](https://app.glassflow.dev) to setup pipelines using the visual editor.  
